In [1]:
import requests as r
import numpy as np
import pandas as pd

# Oanda Packages
from oandapyV20 import API
import oandapyV20
from oandapyV20.contrib.requests import MarketOrderRequest
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.instruments as instruments

from datetime import datetime, timedelta, timezone
import dateutil.parser

In [2]:
class set_up:
    
    # Parameters
    ID='xxxxxxxxxxxxxxxxxxxxxxxxxxx'
    key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
    pair='EUR_USD'
    data_from_oanda = 2000
    length=400
    dev=8
    initial_size=200
    pip_ladder = 5
    params = {
        'count': data_from_oanda,
        'granularity': 'M5'
    }
    
    # Get the OHLC data
    def OHLC(self, which_ohlc):
        self.client = API(access_token=self.key, environment="live")
        self.link = instruments.InstrumentsCandles(instrument=self.pair, params=self.params)
        
        self.ohlc_list = []
        # Call imported user1 class
        self.client.request(self.link)
        self.candles = self.link.response.get("candles")
        for x in range(len(self.candles)):
            self.candleData = self.candles[x].get("mid")[which_ohlc]
            self.ohlc_list.append(self.candleData)
        return self.ohlc_list
    
    # Indicators go here
    def KELCH(self, o, h, l, c, data_from_oanda, length, mult):
        
        #ATR
        self.i = 0  
        self.TR_l = [0]  
        while self.i < len(c)-1:  
            self.TR = max(h[self.i+1], c[self.i]) - min(l[self.i+1], c[self.i])  
            self.TR_l.append(self.TR)  
            self.i = self.i + 1  
        self.TR_s = pd.Series(self.TR_l)
        self.atr = self.TR_s.ewm(span = data_from_oanda, min_periods = length, adjust=False).mean()
        
        #Keltner Channel
        self.KelChM = pd.Series(c).ewm(span=data_from_oanda, min_periods=length, adjust=False).mean()  
        self.KelChU = self.KelChM + mult*self.atr 
        self.KelChD = self.KelChM - mult*self.atr
        
        return self.KelChM, self.KelChU, self.KelChD

In [3]:
class trading:
    
    # state management
    def __init__(self):
        self.status = "Not Trading"
        self.currentTrade = ""
        self.kill = False # <----- This is a kill switch. If it is true, the bot will shut down.
    
     # Entry confirmations
    def enterLong(self):
        if (self.current_price < self. KelChD.values[-1]): return True
        return False

    def enterShort(self): 
        if (self.current_price > self.KelChU.values[-1]): return True
        return False
    
    # Exit confirmation
    def exitLong(self):
        if (self.current_price > self. KelChD.values[-1]): return True
        return False

    def exitShort(self): 
        if (self.current_price < self.KelChU.values[-1]): return True
        return False

    # Check account for how many open trades
    def getTrades(self):
        r = accounts.AccountDetails(set_up.ID)
        client = API(access_token=set_up.key, environment="live")
        rv = client.request(r)
        self.details = rv.get('account')
        return self.details.get('openTradeCount')
    
    # Check size of LONG open trades
    def getSizeTradesLong(self):
        r = accounts.AccountDetails(set_up.ID)
        client = API(access_token=set_up.key, environment="live")
        rv = client.request(r)
        self.details = rv.get('account')
        self.details = self.details.get('positions')[0]['long']['units']
        return float(self.details)
    
    # Check size of SHORT open trades
    def getSizeTradesShort(self):
        r = accounts.AccountDetails(set_up.ID)
        client = API(access_token=set_up.key, environment="live")
        rv = client.request(r)
        self.details = rv.get('account')
        self.details = self.details.get('positions')[0]['short']['units']
        return float(self.details)
    
    # Get entry price of the last trade
    def getPrice(self):
        r = accounts.AccountDetails(set_up.ID)
        client = API(access_token=set_up.key, environment="live")
        rv = client.request(r)
        self.dettagliTrade = rv.get('account')
        self.dettagliTrade = self.dettagliTrade.get('trades')
        self.num=int(len(self.dettagliTrade)-1)
        self.entryPrice = self.dettagliTrade[self.num]["price"]
        return float(self.entryPrice)
    
    # Get entry size of the last trade
    def getLastSize(self):
        r = accounts.AccountDetails(set_up.ID)
        client = API(access_token=set_up.key, environment="live")
        rv = client.request(r)
        self.dettagliTrade = rv.get('account')
        self.dettagliTrade = self.dettagliTrade.get('trades')
        self.num=int(len(self.dettagliTrade)-1)
        self.lastSize = self.dettagliTrade[self.num]["initialUnits"]
        return float(self.lastSize)

    #Define closeout
    def closePosition(self):
        if self.currentTrade == "Long":
            
            data = {"longUnits": "ALL"}
            client = oandapyV20.API(access_token=set_up.key, environment="live")
            r = positions.PositionClose(accountID=set_up.ID,instrument=set_up.pair, data=data)
            client.request(r)
            
        elif self.currentTrade == "Short":
            
            data = {"shortUnits": "ALL"}
            client = oandapyV20.API(access_token=set_up.key, environment="live")
            r = positions.PositionClose(accountID=set_up.ID,instrument=set_up.pair, data=data)
            client.request(r)
            
            
    
     # main trading function
    def main(self):
        
        #initialize data channel
        self.O = list(map(float,set_up().OHLC(which_ohlc='o')))
        self.H = list(map(float,set_up().OHLC(which_ohlc='h')))
        self.L = list(map(float,set_up().OHLC(which_ohlc='l')))
        self.C = list(map(float,set_up().OHLC(which_ohlc='c')))
        self.current_price = self.C[-1]
        #self.data = c.getData()         

        #Initialize Indicators
        self.KelChM, self.KelChU, self.KelChD = set_up().KELCH(o=self.O, h=self.H, l=self.L, c=self.C,
                                                               data_from_oanda=set_up.data_from_oanda,
                                                               length=set_up.length,
                                                               mult=set_up.dev)
        self.currentUKC = self.KelChU.values[-1]
        self.currentLKC = self.KelChD.values[-1]
        self.currentMKC = self.KelChM.values[-1]
        
        # Oanda Parameters
        mktOrderLong = MarketOrderRequest(instrument=set_up.pair,
                      units= set_up.initial_size)
        mktOrderShort = MarketOrderRequest(instrument=set_up.pair,
                       units= (set_up.initial_size *-1))
        
        
        # Trading Conditions
        #################################################################################
        ###########################      NO OPEN TRADES      ############################
        #################################################################################
        # If there are no open trades and the long condition is True
        if self.getTrades() == 0 and self.enterLong() == True:
            
            api = oandapyV20.API(access_token=set_up.key, environment="live")
            r = orders.OrderCreate(set_up.ID, data=mktOrderLong.data)
            api.request(r)
            self.status = "Trading"
            self.currentTrade = "Long"
            print("Trade Executed")
            
        # If there are no open trades and the short condition is True
        elif self.getTrades() == 0 and self.enterShort() == True:
            
            api = oandapyV20.API(access_token=set_up.key, environment="live")
            r = orders.OrderCreate(set_up.ID, data=mktOrderShort.data)
            api.request(r)
            self.status = "Trading"
            self.currentTrade = "Short"
            print("Trade Executed")
                
        # If there are no open trades and the long and short condition is False
        elif self.getTrades() == 0 and self.enterLong() == False and self.enterShort() == False:
            
            print("No Trades Open, Looking for Entry...")

        #################################################################################
        #########################      OPEN TRADES - Short      #########################
        #################################################################################
        
        # If there is at least a short opened and the close condition is True
        elif self.getTrades() > 0 and self.getSizeTradesShort() < 0 and self.exitShort() == True:

            self.closePosition()
            self.status = "Not Trading"
            print("Trade Exited")
            
        # If there is at least a short opened and the price is x pips higher from the last entry
        elif self.getTrades() > 0 and self.getSizeTradesShort() < 0 and self.enterShort() == True \
            and self.getPrice() + set_up.pip_ladder/10000 < self.current_price:
            
            api = oandapyV20.API(access_token=set_up.key, environment="live")
            self.last_entry = self.getLastSize()
            self.units = self.last_entry * 1.25
            mktOrderShort = MarketOrderRequest(instrument=set_up.pair,
                       units=self.units)
            r = orders.OrderCreate(set_up.ID, data=mktOrderShort.data)
            api.request(r)
            print("Trade Executed")
            
        # If there is at least a short opened and the close condition is False
        elif self.getTrades() > 0 and self.getSizeTradesShort() < 0 and self.exitShort() == False:
            
            self.currentTrade = "Short"
            self.status = "Trading"
            print("No exits.. Looking")
            
        #################################################################################
        #########################      OPEN TRADES - Long      #########################
        #################################################################################
            
         # If there is at least a Long opened and the close condition is True
        elif self.getTrades() > 0 and self.getSizeTradesLong() > 0 and self.exitLong()==True:

            self.closePosition()
            self.status = "Not Trading"
            print("Trade Exited")

       # If there is at least a long opened and the price is x pips lower from the last entry
        elif self.getTrades() > 0 and self.getSizeTradesLong() > 0 and self.enterLong() == True \
            and self.getPrice() - set_up.pip_ladder/10000 > self.current_price:
            
            api = oandapyV20.API(access_token=set_up.key, environment="live")
            self.last_entry = self.getLastSize()
            self.units = self.last_entry * 1.25
            mktOrderLong = MarketOrderRequest(instrument=set_up.pair,
                      units=self.units)
            r = orders.OrderCreate(set_up.ID, data=mktOrderLong.data)
            api.request(r)
            print("Trade Executed")
            
        # If there is at least a long opened and the close condition is False
        elif self.getTrades() > 0 and self.getSizeTradesLong() > 0 and self.exitLong()==False:
            
            self.currentTrade = "Long"
            self.status = "Trading"
            print("No exits.. Looking")
                    
        else:
            self.kill = True
            print("Error, Closing down.")

In [ ]:
# Run the bot and kill it if kill switch is engaged
if __name__ == "__main__":
    t = trading()
    while(t.kill == False):
        t.main()